In [126]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import Sequential
from sklearn.metrics import f1_score 
from sklearn.model_selection import train_test_split
from bpmll import bp_mll_loss
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.optimizers.schedules import PolynomialDecay
from sklearn.multioutput import MultiOutputClassifier
from sklearn.utils import shuffle

In [112]:
x = np.load('xinput_train.npy')
y = np.load('labels.npy')

In [631]:
scaler = StandardScaler()?x)

In [113]:
x.shape

(25793, 26613)

In [665]:
np.where(y[:,-1] == 1)[0].shape # how many article do not have prolific authors 
# out of all ~25.7k, only 7k have prolific authors

(18333,)

In [666]:
neg_idx = np.where([y[:,-1] == 1.])[0]
pos_idx = np.where([y[:,-1] != 1.])[0]
# posy_idx = y[np.where([y[:,-1] != 1])[1]]
# negy_idx = y[np.where([y[:,-1] == 1])[1]]

In [667]:
pos = x[pos_idx]
neg = x[neg_idx]
# posy = y[pos_idx]
# negy = y[neg_idx]
posy = y[pos_idx]
negy = y[neg_idx]

In [668]:
# idx = np.random.permutation(int(len(posy)*0.3))
# neg = neg[idx]
# negy = negy[idx]

In [669]:
# rpt_num = int((neg.shape[0] - pos.shape[0])/pos.shape[0])
rpt_pos = np.repeat(pos, 2, axis = 0)

In [670]:
rpt_y = np.repeat(posy, 2,axis = 0)

In [671]:
x = np.concatenate([x, rpt_pos])
y = np.concatenate([y, rpt_y])

# x = np.concatenate([pos, neg])
# y = np.concatenate([posy, negy])


In [114]:
x.shape, y.shape

((25793, 26613), (25793, 101))

In [117]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.5, random_state = 100)

In [116]:
x_train.shape, y_train.shape

((18055, 26613), (18055, 101))

In [6]:
class_w = {}
for i in range(101):
    class_w[i] = 1.
class_w[100] = 0.3

In [94]:

model = Sequential()
model.add(Dense(256, input_dim = x_train.shape[1], activation = 'tanh'))
model.add(Dropout(0.3))
# model.add(Dense(512, activation = 'tanh'))
# model.add(Dropout(0.3))
model.add(Dense(128, activation = 'tanh'))
# model.add(Dropout(0.3))
# model.add(Dense(256, activation = 'tanh'))
model.add(Dense(y_train.shape[1], activation = 'sigmoid'))
lr_schedule = PolynomialDecay(initial_learning_rate = 0.001, decay_steps = 150, end_learning_rate =0.00025, power = 0.5)
model.compile(loss = 'binary_crossentropy', optimizer = 'Adam')
model.fit(x_train, y_train, epochs = 25, batch_size = 128, shuffle = True)

Epoch 1/50
142/142 [==============================] - 8s 41ms/step - loss: 0.0532
Epoch 2/50
142/142 [==============================] - 7s 47ms/step - loss: 0.0300
Epoch 3/50
142/142 [==============================] - 8s 53ms/step - loss: 0.0292
Epoch 4/50
142/142 [==============================] - 7s 52ms/step - loss: 0.0273
Epoch 5/50
142/142 [==============================] - 7s 52ms/step - loss: 0.0256
Epoch 6/50
142/142 [==============================] - 7s 51ms/step - loss: 0.0241
Epoch 7/50
142/142 [==============================] - 7s 52ms/step - loss: 0.0222
Epoch 8/50
142/142 [==============================] - 7s 51ms/step - loss: 0.0194
Epoch 9/50
142/142 [==============================] - 7s 50ms/step - loss: 0.0166
Epoch 10/50
142/142 [==============================] - 7s 51ms/step - loss: 0.0136
Epoch 11/50
142/142 [==============================] - 7s 51ms/step - loss: 0.0110
Epoch 12/50
142/142 [==============================] - 7s 52ms/step - loss: 0.0089
Epoch 13/50
1

In [118]:
pred = model.predict(x_train)
pred = np.where(pred >= 0.5, 1, 0)
f1_score(y_true = y_train, y_pred = pred, average = 'samples')

403/403 [==============================] - 3s 6ms/step


0.9998855311355311

In [119]:
pred_te = model.predict(x_test)
pred_te = np.where(pred_te >= 0.5, 1, 0)
f1_score(y_true = y_test, y_pred = pred_te, average = 'samples')

404/404 [==============================] - 3s 7ms/step


0.8188390064872967

In [98]:
test_data = np.load('xinput_test.npy')

In [99]:
test_data.shape

(800, 26613)

In [86]:
test_data = scaler.transform(test_data)

NameError: name 'scaler' is not defined

In [120]:
prediction = model.predict(test_data)

25/25 [==============================] - 0s 7ms/step


In [121]:
idx = [np.where(p >= 0.5) for p in prediction]

In [122]:
import pandas as pd

In [123]:
df = pd.DataFrame(idx, columns = ['Predict'])

In [124]:
df

,Predict
0,[100]
1,[100]
2,[]
3,[23]
4,[100]
...,...
795,[100]
796,[]
797,[]
798,[]


In [125]:
df.to_csv('PredictALLNN.csv')

In [108]:
svm = LinearSVC(random_state=42, max_iter= 1)
multilabel_classifier = MultiOutputClassifier(svm)
multilabel_classifier = multilabel_classifier.fit(x_train, y_train)

c:\Users\Yi Qian\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\Yi Qian\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\Yi Qian\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\Yi Qian\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\Yi Qian\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iteration

In [109]:
pred = multilabel_classifier.predict(x_test)
pred = np.where(pred >= 0.5, 1, 0)
f1_score(y_true = y_train, y_pred = pred, average = 'samples')

ValueError: Found input variables with inconsistent numbers of samples: [18055, 7738]

In [ ]:
pred_te = model.predict(x_test)
pred_te = np.where(pred_te >= 0.5, 1, 0)
f1_score(y_true = y_test, y_pred = pred_te, average = 'samples')